<a href="https://colab.research.google.com/github/Abidjahan1/sentiment_anlaysis/blob/main/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install kagglehub[pandas-datasets]

from google.colab import files
files.upload()



In [ ]:
#@title dataset uploading with kaggle API
# !mkdir -p ~/.kaggle
# !mv kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [1]:
import kagglehub
import pandas as pd
import re


# Download the dataset using kagglehub
path = kagglehub.dataset_download("kazanova/sentiment140")

print("Dataset files downloaded to:", path)

# Construct the file path to the CSV
csv_path = f"{path}/training.1600000.processed.noemoticon.csv"



Dataset files downloaded to: /kaggle/input/sentiment140


In [ ]:
#@title load csv file with custom names

# Load CSV with custom column names
columns = ['target', 'ids', 'date', 'flag', 'user_name', 'text']

# Load CSV with pandas
df = pd.read_csv(csv_path, encoding="latin-1", header=None,names = columns)
df.head(50)

In [ ]:
 #@title Filter and Clean Data

 # Keep only positive (4) and negative (0) sentiments
df = df[df['target'].isin([0, 4])]

# Map sentiment values: 0 → 0 (negative), 4 → 1 (positive)
df['target'] = df['target'].map({0: 0, 4: 1})

# Drop unused columns
df = df[['text', 'target']]

# Check distribution
df['target'].value_counts()


In [ ]:
#@title Preprocess Text

def clean_text(text):
    text = text.lower()                          # Lowercase all text
    text = re.sub(r'http\S+', '', text)          # Remove URLs
    text = re.sub(r'@\w+', '', text)             # Remove mentions
    text = re.sub(r'#\w+', '', text)             # Remove hashtags
    text = re.sub(r'[^a-z\s]', '', text)         # Remove punctuation/numbers
    return text.strip()

# Apply cleaning to all tweets
df['text'] = df['text'].apply(clean_text)

df.head()


In [ ]:
#@title Better Text Preprocessing

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    words = text.split()
    words = [word for word in words if word not in stop_words]  # remove stopwords
    return " ".join(words)

# Apply cleaning to all tweets
df['text'] = df['text'].apply(clean_text)

df['text']

In [ ]:
#@title Distribution of Sentiment Classes

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import plotly.express as px
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
sns.countplot(data=df, x='target')
plt.title("Sentiment Distribution (0 = Negative, 4 = Positive)")
plt.xlabel("Sentiment")
plt.ylabel("Tweet Count")
plt.show()


In [ ]:
sentiment_count = df['target'].value_counts().reset_index()
sentiment_count.columns = ['Sentiment', 'Count']
sentiment_count['Sentiment'] = sentiment_count['Sentiment'].map({0: "Negative", 4: "Positive"})

fig = px.bar(sentiment_count, x='Sentiment', y='Count',
             color='Sentiment', title="Interactive Sentiment Count")
fig.show()


In [30]:
empty_texts = df['text'].apply(lambda x: len(x.strip()) == 0).sum()
print(f"Empty cleaned texts: {empty_texts}")
# df = df[df['text'].str.strip().astype(bool)]  # Remove rows where text is empty after strip



Empty cleaned texts: 0


In [ ]:
corpus = df['text'].tolist()
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X = vectorizer.fit_transform(corpus)  #
print(corpus[:10])

text_blob = ' '.join(df['text'])
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text_blob)

plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Word Cloud of Cleaned Tweets")
plt.show()


# # Top 20 words for each sentiment
# top_pos = get_top_n_words(df[df['target'] == 1]['text'], 20)
# top_neg = get_top_n_words(df[df['target'] == 0]['text'], 20)

# # Convert to DataFrame for plotting
# pos_df = pd.DataFrame(top_pos, columns=["word", "score"])
# neg_df = pd.DataFrame(top_neg, columns=["word", "score"])

# # Plot
# plt.figure(figsize=(16, 6))
# sns.barplot(data=pos_df, x="score", y="word", color="green")
# plt.title("Top 20 TF-IDF Words - Positive Sentiment")
# plt.show()

# plt.figure(figsize=(16, 6))
# sns.barplot(data=neg_df, x="score", y="word", color="red")
# plt.title("Top 20 TF-IDF Words - Negative Sentiment")
# plt.show()


In [ ]:
#@title  Split into Train and Test Sets
from sklearn.model_selection import train_test_split

X = df['text']
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test


In [ ]:
#@title Text Vectorization with TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

# Create the vectorizer
vectorizer = TfidfVectorizer(max_features=5000)

# Fit on training data and transform both sets
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

'''
TF-IDF converts text into numeric features by calculating:

Term Frequency (TF)

Inverse Document Frequency (IDF)

max_features=5000: keeps the top 5000 most important words.
Adds word pairs (bigrams) like not good, which help detect sarcasm or sentiment more accurately.

'''

In [ ]:
#@title Try Class Balancing
from sklearn.utils import resample

# Separate classes
df_pos = df[df['target'] == 1]
df_neg = df[df['target'] == 0]

# Downsample majority to match minority
df_pos_down = resample(df_pos, replace=False, n_samples=len(df_neg), random_state=42)

# Combine again
df_balanced = pd.concat([df_neg, df_pos_down])


In [ ]:
#@title Train Logistic Regression Model

from sklearn.linear_model import LogisticRegression

# Initialize the model
model = LogisticRegression()

# Train (fit) the model
model.fit(X_train_vec, y_train)

'''Logistic Regression is a linear model for binary classification.

fit() tells the model to learn the relationship between tweet vectors and sentiment labels.'''

In [ ]:
#@title  Evaluate the Model

from sklearn.metrics import accuracy_score, classification_report

# Predict on test data
y_pred = model.predict(X_test_vec)

# Evaluate accuracy and metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

"""
Accuracy: 0.77364375

Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.75      0.77    159494
           1       0.76      0.80      0.78    160506

    accuracy                           0.77    320000
   macro avg       0.77      0.77      0.77    320000
weighted avg       0.77      0.77      0.77    320000

"""


In [ ]:
#@title Save Model and Vectorizer for Django
import pickle

# Save model
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Save TF-IDF vectorizer
with open('tfidf.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)


'''
pickle  save Python objects to reuse later (in Django).

 load model.pkl inside Django to use the trained model.
'''